### Before running:
Please set the file structure as follows: 

* parent folder

    * backgrounds - includes all background pictures

    * dataset - generated images and masks will be placed here 

        * images

        * masks

    * strawberry autolabelling.ipynb

    * strawberry.png

    * strawberry2.png

In [ ]:
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [ ]:
DATASET_PATH = "backgrounds" 
NEW_DATASET_PATH = "dataset"

files = os.listdir(DATASET_PATH)
len(files), files[0]

In [ ]:
MAX_SCALE = 1.5 # Max scale of strawberries during insertation

#### Load strawbery crops

In [ ]:
image_to_insert_options = []

for file in ["strawberry1.png", "strawberry2.png"]:
    strawberry = cv2.imread(file, flags=cv2.IMREAD_UNCHANGED)
    strawberry = cv2.cvtColor(strawberry, cv2.COLOR_BGRA2RGBA)
    strawberry = cv2.rotate(strawberry, cv2.ROTATE_180)
    image_to_insert_options.append(strawberry)
    
image_to_insert_size = 100
x_pad, y_pad = image_to_insert_size * MAX_SCALE, image_to_insert_size * MAX_SCALE # Insertation paddings

#### Modify colors in hsv space

In [ ]:
def change_hsv(img, h_change=0, s_change=0, v_change=0):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
        
    h = cv2.add(h, h_change)
    h[h > 180] -= 180
    h[h < 0] += 180
    
    s = cv2.add(s,s_change)
    s[s > 255] = 255
    s[s < 0] = 0
    
    v = cv2.add(v,v_change)
    v[v > 255] = 255
    v[v < 0] = 0
    
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    
    return img

#### Do augmentations on the image

In [ ]:
def modify_image(image_to_insert):
    red2green = -57 # shift between colors in hue space
    
    # Rotate
    def rotate_image(image, angle):
        image_center = tuple(np.array(image_to_insert.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
        
        return cv2.warpAffine(image_to_insert, rot_mat, image_to_insert.shape[1::-1], flags=cv2.INTER_LINEAR)

    image_to_insert = rotate_image(image_to_insert, angle=np.random.randint(-30, 30))

    # Flip
    if np.random.random() > 0.5:
        image_to_insert = cv2.flip(image_to_insert, 1)

    # Scale
    scale = np.random.random() + 0.5
    image_to_insert_resized = cv2.resize(image_to_insert, None, fx=scale, fy= scale)

    # Take the mask
    mask_strawberry = (image_to_insert_resized[:, :, 3] == 255) 
    image_to_insert_resized = cv2.cvtColor(image_to_insert_resized, cv2.COLOR_RGBA2RGB)

    # Change color of strawberry
    r = np.random.random()
    if r > 0.66:
        # Keep red
        h_change = np.random.randint(-5, 5)
        s_change = np.random.randint(-25, -15)
        v_change = np.random.randint(-25, -15)
    elif r > 0.33:
        # Make half red
        h_change = np.random.randint(red2green + 30, red2green + 40)
        s_change = np.random.randint(-5, 5)
        v_change = np.random.randint(-5, 5)
    else:
        # Make green
        h_change = np.random.randint(red2green + 20, red2green + 25)
        s_change = np.random.randint(-150, -100)
        v_change = np.random.randint(30, 40)

    image_to_insert_resized = change_hsv(image_to_insert_resized, 
                                         h_change=h_change, s_change=s_change, v_change=v_change)

    return image_to_insert_resized, mask_strawberry

In [ ]:
backgrounds = os.listdir(os.path.join(DATASET_PATH))
len(backgrounds)

#### Generate new dataset

In [ ]:
for filename in tqdm(backgrounds):
    
    # Prepare background
    background = cv2.imread(os.path.join(DATASET_PATH, filename))
    background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)

    max_dimension = max(background.shape)
    scale = 1000/max_dimension

    background = cv2.resize(background, None, fx=scale, fy=scale)

    # Find green regions to insert there strawberris
    background_hsv = cv2.cvtColor(background, cv2.COLOR_RGB2HSV)
    mask_green = cv2.inRange(background_hsv, (36, 25, 25), (86, 255,255)) # Green color range (36,25,25) ~ (86, 255,255)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15)) # Clean up the mask
    mask_green_closed = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)
    mask_green_clean = cv2.morphologyEx(mask_green_closed, cv2.MORPH_OPEN, kernel)

    # Get green pixels indices
    X, Y = np.where(mask_green_clean == 255)
    X, Y = zip(*[(x, y) for x, y in zip(X, Y) # Keep pixels within distance from borders
            if 0 < x < background.shape[0] - x_pad and 0 < y < background.shape[1] - y_pad])

    for iteration in range(3): # Use one background image to generate several (3) new images for dataset
        # Choose places to insert strawberries with random
        n_strawberries_to_insert = 5
        pixels_to_insert_ind = np.random.randint(0, len(X), n_strawberries_to_insert)

        # Insert strawberries
        new_image = background.copy()

        segmentation_masks = np.zeros(background.shape[0:2])
        bboxes = []

        for i, ind in enumerate(pixels_to_insert_ind): # Insert every strawberry
            image_to_insert = image_to_insert_options[np.random.randint(0, len(image_to_insert_options))]
            
            # Modify image
            image_to_insert_resized, mask_strawberry = modify_image(image_to_insert)
            
            # Insert crop
            x, y = X[ind], Y[ind]
            image_crop = new_image[x: x + image_to_insert_resized.shape[0], y: y + image_to_insert_resized.shape[1], :]
            image_crop[np.where(mask_strawberry)] = image_to_insert_resized[np.where(mask_strawberry)]

            # Create labels
            segmentation_masks[x: x + image_to_insert_resized.shape[0], 
                               y: y + image_to_insert_resized.shape[1]][mask_strawberry] = i + 1
            
            bboxes.append([x, y, image_to_insert_resized.shape[0], image_to_insert_resized.shape[1]])


        # Save new image and mask
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(NEW_DATASET_PATH, "images", f"{filename}_modified_{iteration}.png"), new_image)

        with open(os.path.join(NEW_DATASET_PATH, "masks", f"{filename}_modified_{iteration}.png"), "wb") as f:
            np.save(f, segmentation_masks.astype(np.int8))
#